In [5]:
import graphviz

In [6]:
dot = graphviz.Digraph(comment='The Round Table')

In [7]:
dot

In [8]:
import graphviz

g = graphviz.Digraph('G', filename='hello.gv')

g.edge('Hello', 'World')

g.view()

'hello.gv.pdf'

## Import `dependencyGraph.dot` file and draw a graph

## Extract all nodes from `DependencyGraph.dot` file

## Find candidates of clashed dependencies

## Prioritise clashed dependencies (manually?)

## For each dependency, assign them same rank and align the related items in ranks